In [7]:
import matplotlib.pyplot as plt
import tensorflow as tf 
import numpy as np 
import os 
from sklearn.perprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'sklearn.perprocessing'